# authentication flow

video guide: [sharepoint link](https://nvidia-my.sharepoint.com/:v:/r/personal/yangj_nvidia_com/Documents/device_auth.mov?csf=1&web=1&e=hsTcLg&nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJTdHJlYW1XZWJBcHAiLCJyZWZlcnJhbFZpZXciOiJTaGFyZURpYWxvZy1MaW5rIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXcifX0%3D)

1. login device flow
2. uri complete to authorize
3. get client_token
4. insert client_token to with renew TTL to redis


device flow guide: https://gitlab-master.nvidia.com/kaizen/auth/starfleet/sf-docs/-/blob/main/apis/user-authentication/device-authorization-endpoint.md

In [1]:
import os

# Need input
CLIENT_IDS = {
    "nonprd": "LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU",
    "prd": "QHWXxZpiDCFZ8T9zCbX9QJUOotq8c6HGVlyYIvDGxXk"
}


# Need input
# user_id will be user for individual person, or could user unique bot id (in future when bot configuration is ready).

userid = ""


In [55]:
# Need input
# option: "stg", "prd"
env = "prd"
print(f"✋Verify that we will retrieve for *{env}* env")
# intentional env overwrite
env = "prd" if env.lower() == "prd" else "nonprd"

✋Verify that we will retrieve for *prd* env


In [56]:
import requests, json

client_id = CLIENT_IDS.get(env)
assert client_id is not None

authorize_url_stg = 'https://stg.login.nvidia.com/device/authorize'
authorize_url_prd = 'https://login.nvidia.com/device/authorize'

authorize_url = authorize_url_stg if env.lower() == "nonprd" else authorize_url_prd

token_url_stg = "https://stg.login.nvidia.com/token"
token_url_prd = "https://login.nvidia.com/token"
token_url = token_url_stg if env.lower() == "nonprd" else token_url_prd

client_token_url_stg = "https://stg.login.nvidia.com/client_token"
client_token_url_prd = "https://login.nvidia.com/client_token"
client_token_url = client_token_url_stg if env.lower() == "nonprd" else client_token_url_prd



In [57]:
device_id = 'thisisadeviceidfornvbotevaluation'
display_name = f"{userid or ''}DeviceFlow"
scope = 'openid profile email'
values_dict = {"device_id": device_id, "client_id": client_id}


### Step 1: get Device Code and Authorize



In [58]:
print("-> authorize_url: ", authorize_url)
print("-> client_id: ", client_id)

-> authorize_url:  https://login.nvidia.com/device/authorize
-> client_id:  QHWXxZpiDCFZ8T9zCbX9QJUOotq8c6HGVlyYIvDGxXk


In [59]:
data = {
    'client_id': client_id,
    'device_id': device_id,
    'display_name': display_name,
    'scope': scope
}

In [60]:
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.post(authorize_url, headers=headers, data=data)
response.text

'{"user_code":"52349250","device_code":"v4RcywSepy1tCobAfm2M7Fiwj66x3FL98RuxjP7Fl0IU9e5kM-bDfRYEJ1APivAfoVXJJ1pw8Lyz_WXjBs3zgQ","verification_uri":"https://static-login.nvidia.com/service/default/pin","verification_uri_complete":"https://static-login.nvidia.com/service/default/pin?user_code=52349250","expires_in":900,"interval":5}'

In [61]:

response_dict = None
if response.status_code == 200:
    response_dict = json.loads(response.text)
    # print("And Type this code", response_dict["user_code"])

else:
    print(f"Invalid request, status code = {response.status_code}, {response.text}")

if response_dict:
    device_code = response_dict["device_code"]
    # print ("device_code: \n", device_code)

print("\nGo to the url endpoint in next code execution block for UI login \n")
print("1. click `Submit` ")
print("2. click `Continue`")
print("3. should see `You are logged in`")


Go to the url endpoint in next code execution block for UI login 

1. click `Submit` 
2. click `Continue`
3. should see `You are logged in`


In [62]:
import webbrowser

webbrowser.open_new_tab(response_dict["verification_uri_complete"])
print("✋Verify complete this steps and able to see `You are logged in`")

✋Verify complete this steps and able to see `You are logged in`


###  Step 2: get Access Token

In [63]:
# URL
print("-> token_url: ", token_url)

-> token_url:  https://login.nvidia.com/token


In [64]:
print("-> device_code: ", device_code)

-> device_code:  v4RcywSepy1tCobAfm2M7Fiwj66x3FL98RuxjP7Fl0IU9e5kM-bDfRYEJ1APivAfoVXJJ1pw8Lyz_WXjBs3zgQ


In [65]:
data = {
    'device_code': device_code,
    'grant_type': 'urn:ietf:params:oauth:grant-type:device_code',
    'client_id': client_id,
}

token_response = requests.post(token_url, headers=headers, data=data)
print (token_response)
if token_response.status_code == 200:
    token_response_dict = token_response.json()
    print("Id token: \n", token_response_dict["id_token"])
    values_dict["id_token"] = token_response_dict["id_token"]
    access_token = token_response_dict["access_token"]
    values_dict["access_token"] = access_token
    print("Access_token: \n", token_response_dict["access_token"])
    print("Expire:\n", token_response_dict["expires_in"])
else:
    print(f"Invalid request, status code = {response_dict.status_code}")


<Response [200]>
Id token: 
 eyJraWQiOiJvYXV0aC1zaWduLWtpZC0yMDIyMDExMjA5Mjk1NSIsImFsZyI6IkVTMjU2In0.eyJzdWIiOiJzc1VWQzE0NnR4cXdsc0xjV2pjdGpVazlNTFpncnJ5Y3hiLUxLMHFKaERjIiwiaWRwX25hbWUiOiJOVi1BRCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJpZHBfaWQiOiJoZ18zS2NhSVhRWXpSbnBua0s4aG9seUhBZllZdUUwNGNEVWliWnprT0R3IiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5udmlkaWEuY29tIiwiZXh0ZXJuYWxfaWQiOiIyZjZjNjZkMS1iMzNlLTRkMTEtOWM1Ni0yZjUxN2IwMzJkYjkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJ5YW5nakBudmlkaWEuY29tIiwic2lkIjoidjRSY3l3U2VweTF0Q29iQWZtMk03Rml3ajY2eDNGTDk4UnV4alA3RmwwSSIsImF1ZCI6WyJRSFdYeFpwaURDRlo4VDl6Q2JYOVFKVU9vdHE4YzZIR1ZseVlJdkRHeFhrIiwiczozRFhUc2pFcjEyTGNQWlB4RE5OWlpteHg5TXA0UkdjajlYVW1zR3VqVXdrIl0sInVwZGF0ZWRfYXQiOjE3Mjg1NDI5MDcsImF6cCI6IlFIV1h4WnBpRENGWjhUOXpDYlg5UUpVT290cThjNkhHVmx5WUl2REd4WGsiLCJuYW1lIjoiSmVzc2ljYSBKaWFvIiwiZXhwIjoxNzI4NTQ2NTIwLCJpYXQiOjE3Mjg1NDI5MjAsImp0aSI6InM3UXlKRUhlUjlpZS00YlJtS3lLaEtLSkJjdzI2blNxVlNpdWZncG4wNFUiLCJlbWFpbCI6InlhbmdqQG52aWRpYS5jb20ifQ.tiaTiBvhIiWrRrVdwZQ4wP0yxSgcby2XK1QKDTNl_Fo_

In [66]:
token_response

<Response [200]>

In [67]:
assert token_response_dict is not None
id_token = token_response_dict['id_token']
# print (f"ID token: {id_token}")


In [68]:
access_token = token_response_dict['access_token']
print(f"Access token: {token_response_dict['access_token']}")


Access token: v4RcywSepy1tCobAfm2M7Fiwj66x3FL98RuxjP7Fl0J1MuowioZfU6IQA1pu0E92bKkskih6mr0CVa2BOhsnRw


## Step 3:  get client token by access token

https://gitlab-master.nvidia.com/kaizen/auth/starfleet/sf-docs/-/blob/main/overview/extras/client-tokens.md

In [69]:
# URL
print("-> client_token_url: ", client_token_url)

-> client_token_url:  https://login.nvidia.com/client_token


In [70]:

headers = {
    # 'Content-Type': 'application/json',
    'Authorization': f"Bearer {access_token}"
}

client_token_response = requests.get(client_token_url, headers=headers)

# Print the response status code and content
print("Response Status Code:", client_token_response.status_code)
print(client_token_response.text)
if client_token_response.status_code == 200:
    client_token_response_dict = json.loads(client_token_response.text)
    


Response Status Code: 200
{"expires_in":7776000,"client_token":"v4RcywSepy1tCobAfm2M7Fiwj66x3FL98RuxjP7Fl0KUmSJq6DEMNhkijiKdwsb_qKEVVxTe9bAC42F5GamNtg"}


In [71]:
assert client_token_response_dict is not None
client_token = client_token_response_dict['client_token']
values_dict["client_token"] = client_token_response_dict["client_token"]
print (f"Client token: {client_token}")
values_dict["client_token_expire_in"] = client_token_response_dict["expires_in"]

Client token: v4RcywSepy1tCobAfm2M7Fiwj66x3FL98RuxjP7Fl0KUmSJq6DEMNhkijiKdwsb_qKEVVxTe9bAC42F5GamNtg


In [72]:
import jwt

decoded_token = jwt.decode(id_token, options={"verify_signature": False})

sub_claim = decoded_token.get("sub")
print("Sub Claim:", sub_claim)

Sub Claim: ssUVC146txqwlsLcWjctjUk9MLZgrrycxb-LK0qJhDc


In [48]:
from datetime import timedelta

def convert_seconds(seconds):
    duration = timedelta(seconds=seconds)
    days = duration.days

    # Calculate total minutes from the duration
    # Note: timedelta.total_seconds() returns the total number of seconds contained in the duration
    total_minutes = duration.total_seconds() / 60

    return days, total_minutes


print(
    f"Access token `{access_token}` expires in {convert_seconds(client_token_response_dict['expires_in'])[0]} days")



Access token `GiVvpLIK2SxxF_s0XC9TnK7_u8dvFF1vk30v-WWtMFuIaUwG-tup6lQVxsJWVSUhpYcWJxBqbzgULH641ehzvA` expires in 90 days


### Step 4: Obtain device auth token from client token

In [53]:
token_url


'https://stg.login.nvidia.com/token'

In [54]:
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

data = {
    "client_token": client_token,
    "grant_type": "urn:ietf:params:oauth:grant-type:client_token",
    'client_id': client_id,
    'sub': sub_claim,
}
device_auth_token_response = requests.post(token_url, headers=headers, data=data)

# Print the response status code and content
print("Response Status Code:", device_auth_token_response.status_code)
if device_auth_token_response.status_code == 200:
    device_auth_token_response_dict = json.loads(device_auth_token_response.text)
print (device_auth_token_response_dict)

Response Status Code: 200
{'access_token': 'GiVvpLIK2SxxF_s0XC9TnK7_u8dvFF1vk30v-WWtMFuCxtIOx0oZx7sCZjY8FbuLvlH3cESxluTDatftxjFBTA', 'token_type': 'Bearer', 'expires_in': 3600, 'client_token': 'GiVvpLIK2SxxF_s0XC9TnK7_u8dvFF1vk30v-WWtMFvYOUTG55Nsg1UDLlMXleLfWOjDPMRvZqYn9GuOxWoTgg', 'id_token': 'eyJraWQiOiJvYXV0aC1zaWduLWtpZC0yMDIyMDEwNDEwMjEyNCIsImFsZyI6IkVTMjU2In0.eyJzdWIiOiJEN1VMdUJmdnpRdmo3ck9lUW5kSlhBRWJWZ1BtQkZWMG5FMzhjSkozMkdVIiwiaWRwX25hbWUiOiJOVi1BRCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJpZHBfaWQiOiIxelU4U0NIWVRSMDR0ZzdVZWRrdExVSGlNTk5UQXZKZ1pOMFk4bmN3bUc4IiwiaXNzIjoiaHR0cHM6Ly9zdGcubG9naW4ubnZpZGlhLmNvbSIsImV4dGVybmFsX2lkIjoiMmY2YzY2ZDEtYjMzZS00ZDExLTljNTYtMmY1MTdiMDMyZGI5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoieWFuZ2pAbnZpZGlhLmNvbSIsInNpZCI6IkdpVnZwTElLMlN4eEZfczBYQzlUbks3X3U4ZHZGRjF2azMwdi1XV3RNRnMiLCJhdWQiOlsiTFJkZFc4ZGFYY1hrSFdfMm52UUJKdDFWOTY5eEhqNm02RUhmLTFTNGVKVSIsInM6NnJybFU2emotb3JMU0dOUlk4WlJOaUFjQkR6M0VSLWJBNGVqMFBQcG5sWSJdLCJ1cGRhdGVkX2F0IjoxNzI4NTQxMDk4LCJhenAiOiJMUmRkVzhkYVhj

In [50]:
{
    "client_token": client_token,
    "grant_type": "urn:ietf:params:oauth:grant-type:client_token",
    'client_id': client_id,
    'sub': sub_claim,
}

{'client_token': 'GiVvpLIK2SxxF_s0XC9TnK7_u8dvFF1vk30v-WWtMFvYOUTG55Nsg1UDLlMXleLfWOjDPMRvZqYn9GuOxWoTgg',
 'grant_type': 'urn:ietf:params:oauth:grant-type:client_token',
 'client_id': 'LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU',
 'sub': 'D7ULuBfvzQvj7rOeQndJXAEbVgPmBFV0nE38cJJ32GU'}

In [ ]:
{'client_token': 'GiVvpLIK2SxxF_s0XC9TnK7_u8dvFF1vk30v-WWtMFuIaUwG-tup6lQVxsJWVSUhpYcWJxBqbzgULH641ehzvA', 'grant_type': 'urn:ietf:params:oauth:grant-type:client_token', 'client_id': 'LRddW8daXcXkHW_2nvQBJt1V969xHj6m6EHf-1S4eJU', 'sub': 'D7ULuBfvzQvj7rOeQndJXAEbVgPmBFV0nE38cJJ32GU'}

In [52]:
assert device_auth_token_response is not None
device_access_token = device_auth_token_response_dict['access_token']
device_token = device_auth_token_response_dict['id_token']
device_client_token = device_auth_token_response_dict['client_token']
device_auth_expires_in = device_auth_token_response_dict['expires_in']

print(f"Device token `{device_token}` expires in {convert_seconds(device_auth_expires_in)[-1]} mins")

values_dict["device_auth_token"] = device_token
values_dict["device_client_token"] = device_client_token
values_dict["device_auth_token_expire_in"] = device_auth_expires_in

Device token `eyJraWQiOiJvYXV0aC1zaWduLWtpZC0yMDIyMDEwNDEwMjEyNCIsImFsZyI6IkVTMjU2In0.eyJzdWIiOiJEN1VMdUJmdnpRdmo3ck9lUW5kSlhBRWJWZ1BtQkZWMG5FMzhjSkozMkdVIiwiaWRwX25hbWUiOiJOVi1BRCIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJpZHBfaWQiOiIxelU4U0NIWVRSMDR0ZzdVZWRrdExVSGlNTk5UQXZKZ1pOMFk4bmN3bUc4IiwiaXNzIjoiaHR0cHM6Ly9zdGcubG9naW4ubnZpZGlhLmNvbSIsImV4dGVybmFsX2lkIjoiMmY2YzY2ZDEtYjMzZS00ZDExLTljNTYtMmY1MTdiMDMyZGI5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoieWFuZ2pAbnZpZGlhLmNvbSIsInNpZCI6IkdpVnZwTElLMlN4eEZfczBYQzlUbks3X3U4ZHZGRjF2azMwdi1XV3RNRnMiLCJhdWQiOlsiTFJkZFc4ZGFYY1hrSFdfMm52UUJKdDFWOTY5eEhqNm02RUhmLTFTNGVKVSIsInM6NnJybFU2emotb3JMU0dOUlk4WlJOaUFjQkR6M0VSLWJBNGVqMFBQcG5sWSJdLCJ1cGRhdGVkX2F0IjoxNzI4NTQxMDk4LCJhenAiOiJMUmRkVzhkYVhjWGtIV18ybnZRQkp0MVY5Njl4SGo2bTZFSGYtMVM0ZUpVIiwibmFtZSI6Ikplc3NpY2EgSmlhbyIsImV4cCI6MTcyODU0NTAxNywiaWF0IjoxNzI4NTQxNDE3LCJqdGkiOiJmSmZxbmJqUi12S0oyMjFteUJqUVFyWDM5ZHFaUDhhRmt4bVVRcEpfNXlnIiwiZW1haWwiOiJ5YW5nakBudmlkaWEuY29tIn0.Lm0JtdWZRVHdoDtgPLKRcrle9OkVnMkW21GtgmLWyDll0aaD-F_Fnq

In [ ]:
import requests

service_url = "https://stgbot-api.nvidia.com/services/" if env.lower() == "nonprd" else "https://nvbot-api.nvidia.com/services/"
headers = {
    'accept': 'application/json',
    'device-initiated': 'false',
    'Authorization': f'Bearer {device_token}'
}

auth_verify_response = requests.get(service_url, headers=headers)

assert auth_verify_response.status_code == 200
print("Verified device token successfully.")


In [ ]:
print("Verified device token successfully.")

In [ ]:
print (f'{values_dict["device_auth_token"]} will expire in {values_dict["device_auth_token_expire_in"] }')


In [ ]:
print (f"Summary: \n{values_dict}")


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Javascript

def copy_to_clipboard(text):
    # Create a button to trigger the copy action
    copy_button = widgets.Button(description=f"Copy device token")
    
    def on_button_click(b):
        # JavaScript code to copy text to clipboard
        js_code = f"""
        var text = `{text}`;
        var temp_input = document.createElement('input');
        document.body.appendChild(temp_input);
        temp_input.value = text;
        temp_input.select();
        document.execCommand('copy');
        document.body.removeChild(temp_input);
        alert('Copied to clipboard: ' + text);
        """
        display(Javascript(js_code))
    
    # Bind the button click event to the function
    copy_button.on_click(on_button_click)
    
    # Display the button
    display(copy_button)

copy_to_clipboard(device_token)

### Complete